In [ ]:
from wsgiref import headers
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
# import pandas as pd # 너무 부담이 큼... 이정도의 프로그램에서 굳이 판다스 필요없음. 레서팬더정도면 충분 :-)
from os import write
import streamlit as st

In [ ]:
class Crawler:
    def __init__(self, url):
        self.base_url = url
        self.visited_url = set()
        self.discovered_url = set()
        self.unavailable_url = set()

    def crawl(self):
        # 만일 베이스 url이 비어있다면??? 그런 상황이 벌어질수 있나??? 없다! 객체를 만들때 필수조건이고 없으면 만들지도 못함.
        # current_url = self.base_url # 여기서 굳이 이런거 할 필요없음... 이건 아래항의 중복.
        self.discovered_url.add(self.base_url) # current_url {} 안에 넣치 않을꺼면 .add() 써야함
        while url_to_visit := self.discovered_url - self.visited_url - self.unavailable_url:
            current_url = url_to_visit.pop()
            # print(current_url)
            header = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
            }
            try:
                res = requests.get(current_url, headers=header, timeout=100)
                # 실패하고 말건 어차피 현재 url은 다신 안돌아갈꺼니깐 
                res.raise_for_status()
                self.visited_url.add(current_url)
            except requests.exceptions.RequestException as e:
                print(f"not able to fetch: {current_url}, with following reason: ")
                self.unavailable_url.add(current_url)
                print(e)
            
            soup = BeautifulSoup(res.content, 'html.parser')
            a_list = soup.select('a')

            for link in a_list:
                url_link = urljoin(self.base_url, link.get('href'))
                self.discovered_url.add(url_link)

            self._dump_result()
            st.write(current_url)
    
    def _dump_result(self):
        url_list = {
            "discovered": self.discovered_url, 
            "visited": self.visited_url, 
            "unavailable": self.unavailable_url,
            }
        for key, val in url_list.items():
            # 나중에 PATH 선택하는 것도 추가
            with open(f'{key}.txt', 'w', encoding='utf-8') as f:
                f.write('\n'.join(val))

    def import_last_result(self, discovered, visited, unavailable):
        try:            
            with open(file=discovered, mode='r', encoding='utf-8') as f:
                self.discovered_url = set(f.read().split('\n'))
            with open(file=visited, mode='r', encoding='utf-8') as f:
                self.visited_url = set(f.read().split('\n'))
            with open(file=unavailable, mode='r', encoding='utf-8') as f:
                self.unavailable_url = set(f.read().split('\n'))
        except Exception as e:
            print(e)

In [ ]:
scraper = Crawler('http://books.toscrape.com')
scraper.crawl()

In [ ]:
scraper.import_last_result('discovered.txt', 'visited.txt', 'unavailable.txt')